In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

In [2]:
df_train= pd.read_csv("../data/processed/train_w_desc.csv")
df_test= pd.read_csv("../data/processed/test_w_desc.csv")
df_valid= pd.read_csv("../data/processed/valid_w_desc.csv")
train_rdkfp= pd.read_csv("../data/processed/train_rdkfp.csv")
valid_rdkfp= pd.read_csv("../data/processed/valid_rdkfp.csv")
test_rdkfp= pd.read_csv("../data/processed/test_rdkfp.csv")

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout, concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Input, Model

In [ ]:
# Reference material on https://pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

In [4]:
#Numerical data aka chemical properties
X_train= df_train[["tpsa","mol_w","qed","HBA","HBD","logP","MR"]]
y_train= df_train["Y"]
X_test= df_test[["tpsa","mol_w","qed","HBA","HBD","logP","MR"]]
y_test= df_test["Y"]
X_valid= df_valid[["tpsa","mol_w","qed","HBA","HBD","logP","MR"]]
y_valid= df_valid["Y"]

In [5]:
from sklearn.preprocessing import MinMaxScaler

In [6]:
scaler= MinMaxScaler()

In [7]:
scaler.fit(X_train)

MinMaxScaler()

In [8]:
X_train_scaled= scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)
X_valid_scaled= scaler.transform(X_valid)

In [9]:
X_train_scaled.shape, X_test_scaled.shape, X_valid_scaled.shape

((28789, 7), (8225, 7), (4113, 7))

In [10]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Recall(name="recall"),
    keras.metrics.AUC(curve="ROC", name="roc-auc")
    ]

In [17]:
Dense??

Init signature:
Dense(
    units,
    activation=None,
    use_bias=True,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs,
)
Source:        
@keras_export("keras.layers.Dense")
class Dense(Layer):
    """Just your regular densely-connected NN layer.

    `Dense` implements the operation:
    `output = activation(dot(input, kernel) + bias)`
    where `activation` is the element-wise activation function
    passed as the `activation` argument, `kernel` is a weights matrix
    created by the layer, and `bias` is a bias vector created by the layer
    (only applicable if `use_bias` is `True`). These are all attributes of
    `Dense`.

    Note: If the input to the layer has a rank greater than 2, then `Dense`
    computes the dot product between the `inputs` and the `kernel` along the
    last axis of the `inputs` a

In [32]:
descriptors_in= Input(shape=(X_train_scaled.shape[-1]), name="descriptors")
dense_a = Dense(8, activation= "relu")(descriptors_in)

fingerprints_in= Input(shape=2048, name= "fingerprints")
dense_b= Dense(32, activation= "sigmoid")(fingerprints_in)

merged= concatenate([dense_a, dense_b])

merged= Dense(64, activation= "relu")(merged)
merged= Dropout(0.3)(merged)
merged= Dense(16, activation= "relu")(merged)
merged= Dropout(0.3)(merged)

drug_class= Dense(1, activation= "sigmoid")(merged)

model= Model([descriptors_in, fingerprints_in], drug_class)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)

checkpoint_path = "checkpoints/weights_func.{epoch:02d}.h5" 
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, monitor='val_roc-auc', save_best_only=True)
callbacks = [EarlyStopping(patience=5), model_checkpoint_callback]

In [11]:
train_inputs = {"descriptors": X_train_scaled, "fingerprints": train_rdkfp} 
valid_inputs =  {"descriptors": X_valid_scaled, "fingerprints": valid_rdkfp}

In [33]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 descriptors (InputLayer)       [(None, 7)]          0           []                               
                                                                                                  
 fingerprints (InputLayer)      [(None, 2048)]       0           []                               
                                                                                                  
 dense_7 (Dense)                (None, 8)            64          ['descriptors[0][0]']            
                                                                                                  
 dense_8 (Dense)                (None, 32)           65568       ['fingerprints[0][0]']           
                                                                                            

In [34]:
history = model.fit(x=train_inputs, y=y_train, validation_data=(valid_inputs, y_valid), callbacks=callbacks, epochs=5, batch_size=16)

Epoch 1/5
1800/1800 [==============================] - 39s 6ms/step - loss: 0.1649 - fn: 1139.0000 - fp: 3.0000 - tp: 7.0000 - recall: 0.0061 - roc-auc: 0.5584 - val_loss: 0.1585 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.6401
Epoch 2/5
1800/1800 [==============================] - 8s 5ms/step - loss: 0.1512 - fn: 989.0000 - fp: 0.0000e+00 - tp: 0.0000e+00 - recall: 0.0000e+00 - roc-auc: 0.6079 - val_loss: 0.1535 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.6413
Epoch 3/5
1800/1800 [==============================] - 9s 5ms/step - loss: 0.1486 - fn: 989.0000 - fp: 0.0000e+00 - tp: 0.0000e+00 - recall: 0.0000e+00 - roc-auc: 0.6317 - val_loss: 0.1534 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.6617
Epoch 4/5
1800/1800 [==============================] - 8s 5ms/step - loss: 0.1482 - fn: 989.0000 - fp: 0.0000e+00 - tp:

In [35]:
history2= model.fit(x=train_inputs, y=y_train, validation_data=(valid_inputs, y_valid), callbacks=callbacks, epochs=20, batch_size=16)

Epoch 1/20
1800/1800 [==============================] - 9s 5ms/step - loss: 0.1414 - fn: 980.0000 - fp: 1.0000 - tp: 9.0000 - recall: 0.0091 - roc-auc: 0.6709 - val_loss: 0.1511 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.6832
Epoch 2/20
1800/1800 [==============================] - 10s 5ms/step - loss: 0.1404 - fn: 978.0000 - fp: 4.0000 - tp: 11.0000 - recall: 0.0111 - roc-auc: 0.6830 - val_loss: 0.1532 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.7091
Epoch 3/20
1800/1800 [==============================] - 9s 5ms/step - loss: 0.1387 - fn: 975.0000 - fp: 10.0000 - tp: 14.0000 - recall: 0.0142 - roc-auc: 0.6957 - val_loss: 0.1428 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.7031
Epoch 4/20
1800/1800 [==============================] - 10s 6ms/step - loss: 0.1284 - fn: 967.0000 - fp: 10.0000 - tp: 22.0000 - recall: 0

In [36]:
#model 2 with Dense layers with more nodes
descriptors_in= Input(shape=(X_train_scaled.shape[-1]), name="descriptors")
dense_a = Dense(8, activation= "relu")(descriptors_in)

fingerprints_in= Input(shape=2048, name= "fingerprints")
dense_b= Dense(32, activation= "sigmoid")(fingerprints_in)

merged= concatenate([dense_a, dense_b])

merged= Dense(128, activation= "relu")(merged)
merged= Dropout(0.3)(merged)
merged= Dense(32, activation= "relu")(merged)
merged= Dropout(0.3)(merged)

drug_class= Dense(1, activation= "sigmoid")(merged)

model2= Model([descriptors_in, fingerprints_in], drug_class)
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)

checkpoint_path = "checkpoints/weights_func2.{epoch:02d}.h5" 
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, monitor='val_recall', save_best_only=True)
callbacks = [EarlyStopping(patience=5), model_checkpoint_callback]

In [37]:
history3= model2.fit(x=train_inputs, y=y_train, validation_data=(valid_inputs, y_valid), callbacks=callbacks, epochs=20, batch_size=16)

Epoch 1/20
1800/1800 [==============================] - 9s 4ms/step - loss: 0.1615 - fn: 1106.0000 - fp: 37.0000 - tp: 40.0000 - recall: 0.0349 - roc-auc: 0.5775 - val_loss: 0.1600 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.6375
Epoch 2/20
1800/1800 [==============================] - 8s 4ms/step - loss: 0.1530 - fn: 989.0000 - fp: 0.0000e+00 - tp: 0.0000e+00 - recall: 0.0000e+00 - roc-auc: 0.5955 - val_loss: 0.1536 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.6454
Epoch 3/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.1498 - fn: 989.0000 - fp: 0.0000e+00 - tp: 0.0000e+00 - recall: 0.0000e+00 - roc-auc: 0.6134 - val_loss: 0.1528 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.6646
Epoch 4/20
1800/1800 [==============================] - 8s 4ms/step - loss: 0.1433 - fn: 989.0000 - fp: 0.0000e+00 

In [38]:
history4= model2.fit(x=train_inputs, y=y_train, validation_data=(valid_inputs, y_valid), callbacks=callbacks, epochs=20, batch_size=16)

Epoch 1/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0978 - fn: 654.0000 - fp: 164.0000 - tp: 335.0000 - recall: 0.3387 - roc-auc: 0.8377 - val_loss: 0.1331 - val_fn: 131.0000 - val_fp: 9.0000 - val_tp: 26.0000 - val_recall: 0.1656 - val_roc-auc: 0.7425
Epoch 2/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.1008 - fn: 697.0000 - fp: 151.0000 - tp: 292.0000 - recall: 0.2952 - roc-auc: 0.8352 - val_loss: 0.1259 - val_fn: 131.0000 - val_fp: 6.0000 - val_tp: 26.0000 - val_recall: 0.1656 - val_roc-auc: 0.7644
Epoch 3/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0993 - fn: 665.0000 - fp: 171.0000 - tp: 324.0000 - recall: 0.3276 - roc-auc: 0.8395 - val_loss: 0.1344 - val_fn: 127.0000 - val_fp: 12.0000 - val_tp: 30.0000 - val_recall: 0.1911 - val_roc-auc: 0.7433
Epoch 4/20
1800/1800 [==============================] - 7s 4ms/step - loss: 0.0964 - fn: 636.0000 - fp: 195.0000 - tp: 353.0000 - recall: 0.3569 - roc-auc: 0.8397

In [12]:
#model 3 with Dense layers with more nodes
descriptors_in= Input(shape=(X_train_scaled.shape[-1]), name="descriptors")
dense_a = Dense(64, activation= "relu")(descriptors_in)

fingerprints_in= Input(shape=2048, name= "fingerprints")
dense_b= Dense(128, activation= "sigmoid")(fingerprints_in)

merged= concatenate([dense_a, dense_b])

merged= Dense(512, activation= "relu")(merged)
merged= Dropout(0.3)(merged)
merged= Dense(256, activation= "relu")(merged)
merged= Dropout(0.3)(merged)
merged= Dense(64, activation= "relu")(merged)
merged= Dropout(0.3)(merged)

drug_class= Dense(1, activation= "sigmoid")(merged)

model3= Model([descriptors_in, fingerprints_in], drug_class)
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)

checkpoint_path = "checkpoints/weights_func2.{epoch:02d}.h5" 
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, monitor='val_recall', save_best_only=True)
callbacks = [EarlyStopping(patience=5), model_checkpoint_callback]

In [13]:
history5= model3.fit(x=train_inputs, y=y_train, validation_data=(valid_inputs, y_valid), callbacks=callbacks, epochs=20, batch_size=16)

Epoch 1/20
1800/1800 [==============================] - 24s 12ms/step - loss: 0.1616 - fn: 989.0000 - fp: 6.0000 - tp: 0.0000e+00 - recall: 0.0000e+00 - roc-auc: 0.5618 - val_loss: 0.1616 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.6494
Epoch 2/20
1800/1800 [==============================] - 22s 12ms/step - loss: 0.1528 - fn: 989.0000 - fp: 0.0000e+00 - tp: 0.0000e+00 - recall: 0.0000e+00 - roc-auc: 0.6060 - val_loss: 0.1538 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.6596
Epoch 3/20
1800/1800 [==============================] - 24s 13ms/step - loss: 0.1512 - fn: 989.0000 - fp: 0.0000e+00 - tp: 0.0000e+00 - recall: 0.0000e+00 - roc-auc: 0.6168 - val_loss: 0.1502 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.6921
Epoch 4/20
1800/1800 [==============================] - 24s 13ms/step - loss: 0.1481 - fn: 989.0000 - fp

In [14]:
history6= model3.fit(x=train_inputs, y=y_train, validation_data=(valid_inputs, y_valid), callbacks=callbacks, epochs=20, batch_size=16)

Epoch 1/20
1800/1800 [==============================] - 23s 13ms/step - loss: 0.1024 - fn: 728.0000 - fp: 159.0000 - tp: 261.0000 - recall: 0.2639 - roc-auc: 0.8385 - val_loss: 0.1208 - val_fn: 106.0000 - val_fp: 20.0000 - val_tp: 51.0000 - val_recall: 0.3248 - val_roc-auc: 0.7790
Epoch 2/20
1800/1800 [==============================] - 25s 14ms/step - loss: 0.1324 - fn: 899.0000 - fp: 38.0000 - tp: 90.0000 - recall: 0.0910 - roc-auc: 0.7130 - val_loss: 0.1592 - val_fn: 144.0000 - val_fp: 10.0000 - val_tp: 13.0000 - val_recall: 0.0828 - val_roc-auc: 0.7319
Epoch 3/20
1800/1800 [==============================] - 22s 12ms/step - loss: 0.1345 - fn: 950.0000 - fp: 27.0000 - tp: 39.0000 - recall: 0.0394 - roc-auc: 0.7062 - val_loss: 0.1465 - val_fn: 157.0000 - val_fp: 0.0000e+00 - val_tp: 0.0000e+00 - val_recall: 0.0000e+00 - val_roc-auc: 0.7336
Epoch 4/20
1800/1800 [==============================] - 23s 13ms/step - loss: 0.1319 - fn: 921.0000 - fp: 26.0000 - tp: 68.0000 - recall: 0.0688 - r